In [14]:
"""
Calculating the fluxes and uncertainties within an aperture for ALMA maps
"""
from photutils.aperture import CircularAperture, EllipticalAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.io import fits
import numpy as np
from astropy.cosmology import Planck15

ra = '13:11:29.935' #coordinates of your galaxy (in SkyCoord accepted format)
dec = '-1:19:18.750'
smaj = 2.5 #arcseconds #major and minor axis dimensions and orientation angle of elliptical aperture
smin = 1.5 #arcseconds
ang = -np.pi/6 #radians

def image_reader(fits_file_path):
    """
    Read a fits file and return data and header separately.
    
    If data contains axes of length 1, remove these axes.
    """    
    im = fits.open(fits_file_path)
    header = im[0].header
    array2d = np.squeeze(im[0].data) #remove axes of length 1
    return(array2d, header)

def aper_pos(header):
    """
    Use header to get the WCS.
    
    Convert given aperture RA and Dec position into pixel x,y position.
    
    Return x, y in pixels, and pixel size in degrees.
    (assuming pixel size information is present in the header).
    """
    w = WCS(header,naxis=2)
    center = SkyCoord(ra, dec, unit=(u.hourangle, u.deg),frame='fk5')
    x,y = w.world_to_pixel(center)
    pix_len = header['CDELT2']
    return(x,y,pix_len)

def beam_area_pix(header):
    """
    Get beam properties and pixel size from header.
    
    Return beam size in pixels.
    """    
    pix_len = header['CDELT2']
    beam_maj = header['BMAJ']
    beam_min = header['BMIN']
    beam_area_deg = (np.pi / (4*np.log(2.))) * beam_maj*beam_min
    npix_per_beam = beam_area_deg/pix_len**2
    return(npix_per_beam)
    
def flux_calculator(array2d, header, aper=None):
    """
    Calculate flux (in Jy or Jy-km/s if moment-0 map)
    within given aperture centered at given RA and Dec coordinates.
    
    If no aperture is given, use a 2.5"x1.5" elliptical aperture oriented at -30 degrees.
    """
    if aper is None:
        galx,galy,pix_len = aper_pos(header)
        aper = EllipticalAperture((galx,galy), a = smaj/(pix_len*3600), b = smin/(pix_len*3600), theta = ang) #arcsec to pixel conversion
        
    tab = aperture_photometry(array2d, apertures=aper)
    
    flux_per_beam = tab['aperture_sum'][0]
    npix_per_beam = beam_area_pix(header)
    flux_tot = flux_per_beam/npix_per_beam
    
    return(flux_tot) #Jy not Jy/beam

def luminosity(array2d, header, nu_obs=233, z=7, aper=None):
    """
    Calculate luminosity from flux using Eq.18 from Casey et al 2014.
    
    Observed frequency (nu_obs) and redshift (z) must be provided;
    defaults are 233GHz and 7 respectively.
    
    Return luminosity in multiples of solar luminosity.
    """
    D_L = Planck15.luminosity_distance(z).value #calculate luminosity distance from redshift
    S_nu_kmps = flux_calculator(array2d, header, aper=aper)
    L = 1.04e-3 * S_nu_kmps * nu_obs * (D_L**2) # L_sun
    return(L)

def find_ratio(arr1, hed1, arr2, hed2, nu_obs1=233, nu_obs2=233, flux_or_lum='flux', aper=None):
    """
    Calculate ratio of fluxes or luminosities (specify using flux_or_lum parameter)
    between two maps at given RA and Dec position.
    
    The observed frequencies of each line (nu_ob1 and nu_obs2)
    must be provided to calculate luminosity ratio.
    """
    f1 = flux_calculator(arr1, hed1, aper=aper)
    f2 = flux_calculator(arr2, hed2, aper=aper)
    ratio = f1/f2
    if flux_or_lum == 'lum':
        ratio = ratio*(nu_obs1/nu_obs2)
    return(ratio)

def stdev_annulus(array2d, header, annulus_radii=(3,4)):
    """
    Make an annulus with given inner and outer radii (in pixels) around given RA and Dec position.
    
    Default radii are 3 and 4 arcseconds;
    
    Calculate standard deviation within the annulus.
    """
    galx,galy,pix_len = aper_pos(header)
    rin, rout = annulus_radii
    
    nx, ny = array2d.shape
    Y, X = np.ogrid[0:nx, 0:ny] # Y is a column and X is a row
        
    dist_from_gal_center = np.sqrt((X - galx)**2 + (Y - galy)**2)
    masked_image = np.where(((rin/(pix_len*3600) <= dist_from_gal_center) & (dist_from_gal_center <= rout/(pix_len*3600))), array2d, np.nan)  
    stdev = np.nanstd(masked_image)
    return(stdev)

def uncertainty(array2d, header, nu_obs=233, flux_or_lum='flux', annulus_radii=(3,4), aper=None):
    """
    Calculate standard deviation within annulus around given RA and Dec position.
    This is the noise level in the image.
    
    Scale this by the square root of (aperture area/beam area) to get the total noise within the aperture.
    """
    if aper is None:
        galx,galy,pix_len = aper_pos(header)
        aper = EllipticalAperture((galx,galy), a = smaj/(pix_len*3600), b = smin/(pix_len*3600), theta = ang) #arcsec to pixel conversion
        
    stdev = stdev_annulus(array2d, header, annulus_radii=annulus_radii)
#     print(stdev)
    npix_per_beam = beam_area_pix(header) # beam area in pixels == number of pixels per beam
    nbeams_aper = (aper.area)/npix_per_beam
    uncert = stdev*np.sqrt(nbeams_aper)
    
    if flux_or_lum == 'lum':
        uncert = 1.04e-3 * uncert * nu_obs * (D_L**2)
    
    return(uncert)

def ratio_uncertainty(arr1, hed1, arr2, hed2, nu_obs1=233, nu_obs2=233, flux_or_lum='flux', annulus_radii=(3,4), aper=None):
    """
    Calculate the fluxes and uncertainties within the given aperture for both images.
    
    Calculate the ratio of fluxes and uncertainties.
    
    Propagate the uncertainties through the ratio.
    
    Return the uncertainty on the ratio.
    """
    if flux_or_lum == 'flux':
        q1 = flux_calculator(arr1, hed1, aper=aper)
        q2 = flux_calculator(arr2, hed2, aper=aper)
        
    if flux_or_lum == 'lum':
        q1 = luminosity(arr1, hed1, nu_obs=nu_obs1, aper=aper)
        q2 = luminosity(arr2, hed2, nu_obs=nu_obs2, aper=aper)
    
    qr = q1/q2
        
    sig1 = uncertainty(arr1, hed1, nu_obs=nu_obs1, flux_or_lum=flux_or_lum, annulus_radii=annulus_radii, aper=aper)
    sig2 = uncertainty(arr2, hed2, nu_obs=nu_obs2, flux_or_lum=flux_or_lum, annulus_radii=annulus_radii, aper=aper)
        
    sigr = qr * np.sqrt( (sig1 / q1)**2 + (sig2 / q2)**2 )
    
    return(sigr)